<a href="https://colab.research.google.com/github/osgeokr/GEE-PAM-Book/blob/main/2%EC%9E%A5_%EC%96%B4%EC%8A%A4%EC%97%94%EC%A7%84_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2장. 어스엔진 데이터셋 실습



In [ ]:
import ee
import geemap

import pandas as pd

In [ ]:
# Earth Engine 인증
ee.Authenticate()

# Earth Engine 초기화
ee.Initialize(project='ee-foss4g')

## 2.1. 피처 컬렉션

### 2.1.1. 피처 컬렉션 사용 (보호지역)

In [ ]:
# 피처 컬렉션 객체 (보호지역)
wdpa = ee.FeatureCollection("WCMC/WDPA/current/polygons")

In [ ]:
# 피처 선택 (보호지역 ID == 32712)
wdpa_id = 30712
protected_area = wdpa.filter(ee.Filter.eq('WDPAID', wdpa_id))

# 피처 이름 (변산반도)
area_name = protected_area.first().get('NAME').getInfo()
print("Name:", area_name)

In [ ]:
# 피처 컬렉션 출력 (변산반도)
m = geemap.Map(layout={'height':'400px', 'width':'800px'},
                 basemap='Esri.WorldImagery')

m.addLayer(protected_area, {'color': 'green'}, area_name)
m.centerObject(protected_area, 11)
m

### 2.1.2. 피처 컬렉션 사용 (행정구역)

In [ ]:
# 피처 컬렉션 객체 (행정구역)
countries = ee.FeatureCollection("FAO/GAUL/2015/level0")

# 피처 선택 ('ADM0_NAME' == '대한민국)
kr = countries.filter(ee.Filter.eq('ADM0_NAME', 'Republic of Korea'))

# 피처 컬렉션 출력 (대한민국)
m = geemap.Map(layout={'height':'400px', 'width':'800px'})
m.addLayer(kr, {}, "Republic of Korea")
m.centerObject(kr, 6)
m

## 2.2. 이미지 컬렉션

### 2.2.1. 이미지 사용 (DEM)

In [ ]:
# 이미지 객체 (DEM)
srtm = ee.Image("USGS/SRTMGL1_003")
elevation = srtm.select("elevation")
vis_params = {
    "min": 0,
    "max": 5000,
    "palette": [
        "000000",
        "006600",
        "009900",
        "33CC00",
        "996600",
        "CC9900",
        "CC9966",
        "FFFFFF"
    ]
}

# 이미지 출력 (DEM)
m = geemap.Map(layout={"height": "400px", "width": "800px"})
m.addLayer(elevation, vis_params, "Elevation")
m.centerObject(kr, 4)
m

### 2.2.2. 이미지 컬렉션 (수목 피복률)

In [ ]:
# 이미지 컬렉션 객체 (MOD44B)
modis = ee.ImageCollection('MODIS/006/MOD44B')

# 이미지 선택 (수목 피복률)
filtered_modis = modis.filterDate('2003-01-01', '2020-12-31')
selected_modis = filtered_modis.select('Percent_Tree_Cover')
median_ptc = selected_modis.median()
vis_params = {
    'min': 0,
    'max': 100,
    'palette': ['bbe029', '0a9501', '074b03']}

# 이미지 출력 (수목 피복률)
m = geemap.Map(layout={"height": "400px", "width": "800px"})
m.addLayer(median_ptc, vis_params, "Percent Tree Cover")
m.centerObject(kr, 4)
m

### 2.2.3. 이미지 컬렉션 (토지피복지도)

In [ ]:
# 이미지 컬렉션 객체 (Sentinel-2 이미지)
image = geemap.dynamic_world_s2(protected_area.geometry(),
                                '2022-01-01', '2023-01-01')
vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}

# 이미지 출력 (Sentinel-2 이미지)
m = geemap.Map(layout={"height": "400px", "width": "800px"})
m.addLayer(image, vis_params, "Sentinel-2 image")
m.centerObject(protected_area, 11)
m

In [ ]:
landcover = geemap.dynamic_world(protected_area.geometry(),
                                 '2022-01-01', '2023-01-01')

# 이미지 출력 (토지피복지도)
m = geemap.Map(layout={"height": "400px", "width": "800px"})
m.addLayer(landcover, {}, 'Land Cover')
m.add_legend(title="Dynamic World Land Cover", builtin_legend='Dynamic_World')
m.centerObject(protected_area, 11)
m

## 2.3. 타입랩스 생성

### 2.3.1. 패키지 추가 설치

In [ ]:
!pip install -q -U ffmpeg-python
!pip install -q -U geokakao

In [ ]:
import geokakao as gk
import geopy.distance as gd

### 2.3.2. 타임랩스 영역


In [ ]:
data = {'Name': ['새만금국가산업단지'],
        'Address': ['전북특별자치도 군산시 오식도동 1100']}
df = pd.DataFrame(data)
gk.add_coordinates_to_dataframe(df, 'Address')

df.head()

In [ ]:
from ipyleaflet import TileLayer

# Vworld 영상지도 객체
vworld_satellite = TileLayer(
    url="https://xdworld.vworld.kr/2d/Satellite/service/{z}/{x}/{y}.jpeg",
    name="Vworld Satellite",
    attribution="Vworld",
)
# Vworld 하이브리드지도 객체
vworld_hybrid = TileLayer(
    url="https://xdworld.vworld.kr/2d/Hybrid/service/{z}/{x}/{y}.png",
    name="Vworld Hybrid",
    attribution="Vworld",
)

m = geemap.Map(width="800px", height="400px")
m.add_layer(vworld_satellite)
m.add_layer(vworld_hybrid)

# 새만금국가산업단지 위치 가시화
latitude, longitude = df.loc[0, ["decimalLatitude", "decimalLongitude"]].astype(float)
location = ee.Geometry.Point([longitude, latitude])
m.centerObject(location, 13)  # 지도의 중심 설정
m  # 지도 객체 출력

In [ ]:
def calculate_bbox(longitude, latitude, width, height):
    # 중심 좌표
    center = (latitude, longitude)

    # 반쪽 폭과 높이 계산 (미터)
    half_width = width / 2
    half_height = height / 2

    # 경계 좌표 계산
    north = gd.distance(meters=half_height).destination(center, 0).latitude
    south = gd.distance(meters=half_height).destination(center, 180).latitude
    east = gd.distance(meters=half_width).destination(center, 90).longitude
    west = gd.distance(meters=half_width).destination(center, 270).longitude

    # 경계 상자 반환
    bbox = ee.Geometry.BBox(west, south, east, north)
    return bbox

# 타임랩스 영역
width = 4000; height = 2000
roi = calculate_bbox(longitude, latitude, width, height)

### 2.3.3. 타임랩스 생성

In [ ]:
# 센티널 1호 타임랩스 생성
timelapse = geemap.sentinel1_timelapse(
    roi,
    out_gif="sentinel1.gif",
    start_year=2019,
    end_year=2023,
    start_date="01-01",
    end_date="12-31",
    frequency="month",
    vis_params={"min": -30, "max": 0},
    palette="Greys",
    frames_per_second=3,
    title="Sentinel-1 Timelapse",
    add_colorbar=True,
    colorbar_bg_color="gray",
)
geemap.show_image(timelapse)

In [ ]:
# 센티널 2호 타임랩스 생성
timelapse = geemap.sentinel2_timelapse(
    roi,
    out_gif='sentinel2.gif',
    start_year=2018,
    end_year=2023,
    start_date='01-01',
    end_date='12-31',
    frequency='year',
    bands=['SWIR1', 'NIR', 'Red'],
    frames_per_second=3,
    title='Sentinel-2 Timelapse',
)
geemap.show_image(timelapse)

In [ ]:
from google.colab import files

files.download('sentinel1.gif')
files.download('sentinel2.gif')